# Cleaning data and the skies

## 📖 Background
Your are a data analyst at an environmental company. Your task is to evaluate ozone pollution across various regions.

You’ve obtained data from the U.S. Environmental Protection Agency (EPA), containing daily ozone measurements at monitoring stations across California. However, like many real-world datasets, it’s far from clean: there are missing values, inconsistent formats, potential duplicates, and outliers.

Before you can provide meaningful insights, you must clean and validate the data. Only then can you analyze it to uncover trends, identify high-risk regions, and assess where policy interventions are most urgently needed.

## 💾 The data
The data is a modified dataset from the U.S. Environmental Protection Agency ([EPA](https://www.epa.gov/outdoor-air-quality-data/download-daily-data)). 

#### Ozone contains the daily air quality summary statistics by monitor for the state of California for 2024. Each row contains the date and the air quality metrics per collection method and site
- "Date" - the calendar date with which the air quality values are associated
- "Source" - the data source: EPA's Air Quality System (AQS), or Airnow reports
- "Site ID" - the id for the air monitoring site
- "POC" - the id number for the monitor
- "Daily Max 8-hour Ozone Concentration" - the highest 8-hour value of the day for ozone concentration
- "Units" - parts per million by volume (ppm)
- "Daily AQI Value" - the highest air quality index value for the day, telling how clean or polluted the air is (a value of 50 represents good air quality, while a value above 300 is hazardous)
- "Local Site Name" - name of the monitoring site
- "Daily Obs Count" - number of observations reported in that day
- "Percent Complete" - indicates whether all expected samples were collected
- "Method Code" - identifier for the collection method
- "CBSA Code" - identifier for the core base statistical area (CBSA)
- "CBSA Name" - name of the core base statistical area
- "State FIPS Code" - identifier for the state
- "State" - name of the state
- "County FIPS Code" - identifer for the county
- "County" - name of the county
- "Site Latitude" - latitude coordinates of the site
- "Site Longitude" - longitude coordinates of the side

In [75]:
import pandas as pd
import numpy as np
import re
from datetime import date

ozone = pd.read_csv('data/ozone.csv')
ozone.head()

,Date,Source,Site ID,POC,Daily Max 8-hour Ozone Concentration,Units,Daily AQI Value,Local Site Name,Daily Obs Count,Percent Complete,Method Code,CBSA Code,CBSA Name,County FIPS Code,County,Site Latitude,Site Longitude
0,/2024,AQS,60010007,1,0.031,ppm,29.0,Livermore,17,100.0,47.0,41860.0,"San Francisco-Oakland-Hayward, CA",1,Alameda,37.687526,-121.784217
1,01/02/2024,AQS,60010007,1,0.037,ppm,34.0,Livermore,17,100.0,47.0,41860.0,"San Francisco-Oakland-Hayward, CA",1,Alameda,37.687526,-121.784217
2,/2024,AQS,60010007,1,NaN,ppm,30.0,Livermore,17,100.0,47.0,41860.0,"San Francisco-Oakland-Hayward, CA",1,Alameda,37.687526,-121.784217
3,January 04/2024,AQS,60010007,1,0.026,ppm,24.0,Livermore,17,100.0,47.0,41860.0,"San Francisco-Oakland-Hayward, CA",1,Alameda,37.687526,-121.784217
4,January 05/2024,AQS,60010007,1,0.027,ppm,25.0,Livermore,17,100.0,47.0,41860.0,"San Francisco-Oakland-Hayward, CA",1,Alameda,37.687526,-121.784217


## 💪 Competition challenge

Create a report that covers the following:
1. Your EDA and data cleaning process.
2. How does daily maximum 8-hour ozone concentration vary over time and regions?
3. Are there any areas that consistently show high ozone concentrations? Do different methods report different ozone levels?
4. Consider if urban activity (weekend vs. weekday) has any affect on ozone levels across different days.
5. Bonus: plot a geospatial heatmap showing any high ozone concentrations.

## 🧑‍⚖️ Judging criteria

| CATEGORY | WEIGHTING | DETAILS                                                              |
|:---------|:----------|:---------------------------------------------------------------------|
| **Recommendations** | 35%       | <ul><li>Clarity of recommendations - how clear and well presented the recommendation is.</li><li>Quality of recommendations - are appropriate analytical techniques used & are the conclusions valid?</li><li>Number of relevant insights found for the target audience.</li></ul>       |
| **Storytelling**  | 35%       | <ul><li>How well the data and insights are connected to the recommendation.</li><li>How the narrative and whole report connects together.</li><li>Balancing making the report in-depth enough but also concise.</li></ul> |
| **Visualizations** | 20% | <ul><li>Appropriateness of visualization used.</li><li>Clarity of insight from visualization.</li></ul> |
| **Votes** | 10% | <ul><li>Up voting - most upvoted entries get the most points.</li></ul> |

## ✅ Checklist before publishing into the competition
- Rename your workspace to make it descriptive of your work. N.B. you should leave the notebook name as notebook.ipynb.
- **Remove redundant cells** like the judging criteria, so the workbook is focused on your story.
- Make sure the workbook reads well and explains how you found your insights. 
- Try to include an **executive summary** of your recommendations at the beginning.
- Check that all the cells run without error

## ⌛️ Time is ticking. Good luck!

## 1. Cleaning Data

### 2.1 Date columns

In [77]:
ozone.columns

Index(['Date', 'Source', 'Site ID', 'POC',
       'Daily Max 8-hour Ozone Concentration', 'Units', 'Daily AQI Value',
       'Local Site Name', 'Daily Obs Count', 'Percent Complete', 'Method Code',
       'CBSA Code', 'CBSA Name', 'County FIPS Code', 'County', 'Site Latitude',
       'Site Longitude'],
      dtype='object')

I found out that the data were supposed to be recorded daily, but there are gaps, duplicates, or inconsistencies in the date column.

In [95]:
def reorder_date(index_start, index_end, stardate, endate):
    ozone.loc[index_start:index_end, 'Date'] = pd.date_range(start=stardate, end=endate)
    ozone['Date'] = pd.to_datetime(ozone['Date']).dt.date

reorder_date(0, 23, '2024-01-01', '2024-01-24')

In [96]:
reorder_date(25, 55, '2024-01-01', '2024-01-31')

In [ ]:
reorder_date(56, 84, '2024-02-01', '2024-02-29')

In [124]:
reorder_date(116, 144, '2024-04-02', '2024-04-30')
ozone.iloc[170:190, :]

,Date,Source,Site ID,POC,Daily Max 8-hour Ozone Concentration,Units,Daily AQI Value,Local Site Name,Daily Obs Count,Percent Complete,Method Code,CBSA Code,CBSA Name,County FIPS Code,County,Site Latitude,Site Longitude
170,2024-06-29,AirNow,60010009,1,0.029,ppm,27.0,Oakland,24,100.0,NaN,41860.0,"San Francisco-Oakland-Hayward, CA",1,Alameda,37.743065,-122.169935
171,2024-06-30,AirNow,60010009,1,0.018,ppm,17.0,Oakland,24,100.0,NaN,41860.0,"San Francisco-Oakland-Hayward, CA",1,Alameda,37.743065,-122.169935
172,2024-01-01,AirNow,60010009,1,0.019,ppm,18.0,Oakland,24,100.0,NaN,41860.0,"San Francisco-Oakland-Hayward, CA",1,Alameda,37.743065,-122.169935
173,2024-07-02,AirNow,60010009,1,0.044,ppm,41.0,Oakland,24,100.0,NaN,41860.0,"San Francisco-Oakland-Hayward, CA",1,Alameda,37.743065,-122.169935
174,2024-01-01,AirNow,60010009,1,0.043,ppm,40.0,Oakland,24,100.0,NaN,41860.0,"San Francisco-Oakland-Hayward, CA",1,Alameda,37.743065,-122.169935
175,2024-01-01,AirNow,60010009,1,0.038,ppm,35.0,Oakland,24,100.0,NaN,41860.0,"San Francisco-Oakland-Hayward, CA",1,Alameda,37.743065,-122.169935
176,2024-07-05,AirNow,60010009,1,0.032,ppm,30.0,Oakland,24,100.0,NaN,41860.0,"San Francisco-Oakland-Hayward, CA",1,Alameda,37.743065,-122.169935
177,2024-07-06,AirNow,60010009,1,0.034,ppm,31.0,Oakland,24,100.0,NaN,41860.0,"San Francisco-Oakland-Hayward, CA",1,Alameda,37.743065,-122.169935
178,2024-07-07,AirNow,60010009,1,0.030,ppm,28.0,Oakland,24,100.0,NaN,41860.0,"San Francisco-Oakland-Hayward, CA",1,Alameda,37.743065,-122.169935
179,2024-07-08,AirNow,60010009,1,NaN,ppm,25.0,Oakland,24,100.0,NaN,41860.0,"San Francisco-Oakland-Hayward, CA",1,Alameda,37.743065,-122.169935


In [ ]:
# def parse_strict_date(val):
#     val = str(val).strip()

#     parts = [p for p in re.split(r'[-/\s]', val) if p]
#     if len(parts) != 3:
#         return pd.NaT
    
#     try:
#         return pd.to_datetime(val)
#     except:
#         return pd.NaT

# ozone['Date'] = ozone['Date'].apply(parse_strict_date)

In [ ]:
miss_value = ozone.isnull().sum()
miss_value_percentage = 100 * miss_value / len(ozone)
miss_table = pd.concat([miss_value, miss_value_percentage], axis=1)
miss_table

,0,1
Date,0,0.000000
Source,0,0.000000
Site ID,0,0.000000
POC,0,0.000000
Daily Max 8-hour Ozone Concentration,2738,5.000091
Units,0,0.000000
Daily AQI Value,2738,5.000091
Local Site Name,0,0.000000
Daily Obs Count,0,0.000000
Percent Complete,0,0.000000
